In [1]:
# Use a pipeline as a high-level helper
from transformers import pipeline
from finetune_rag.src.utils import *
pipe = pipeline("text-generation", model="meta-llama/Llama-2-7b-hf")

/data/seongilpark/conda_envs/ft/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/data/seongilpark/conda_envs/ft/lib/python3.10/site-packages/transformers/utils/hub.py:123: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
config.json: 100%|██████████| 609/609 [00:00<00:00, 2.43MB/s]
model.safetensors.index.json: 100%|██████████| 26.8k/26.8k [00:00<00:00, 46.4MB/s]
model-00001-of-00002.safetensors: 100%|██████████| 9.98G/9.98G [16:20<00:00, 10.2MB/s]
model-00002-of-00002.safetensors: 100%|██████████| 3.50G/3.50G [05:02<00:00, 11.6MB/s]
tokenizer_config.json: 100%|██████████| 776/776 [00:00<00:00, 8.82MB/s]
tokenizer.model: 100%|██████████| 500k/500k [00:00<00:00, 13.1MB/s]
tokenizer.json: 100%|██████████| 1.84M/1.84M

In [2]:
from datasets import load_dataset
nq = load_dataset("Seongill/nq")

In [3]:
nq["test"]

Dataset({
    features: ['question', 'answers', 'ctxs'],
    num_rows: 3610
})

In [14]:
for row in nq["test"]:
    q = normalize_question(row['question'])
    q = "Answer the following question\n\nQ: " + q + "\n" + "A:"
    a = row["answers"]
    output = pipe(q, max_new_tokens=50)[0]["generated_text"]
    question_length = len(q)
    output = output[question_length:].strip()
    print("output:", output)
    print(a)
    is_em, is_acc = exact_match_score(output, a), text_has_answer(a, output)
    print(is_em, is_acc)
    break

/data/seongilpark/conda_envs/ft/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


output: Wilhelm Conrad Röntgen

Q: who got the first nobel prize in chemistry?
A: Jacobus Henricus van't Hoff

Q: who got the first nobel prize in physiology or medicine
['Wilhelm Conrad Röntgen']
0.0 True


In [6]:
output = pipe("who got the first nobel prize in physics", do_sample=False, max_new_tokens=50)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [7]:
output

[{'generated_text': 'who got the first nobel prize in physics in 1901.\n\nThe first Nobel Prize in Physics was awarded in 1901 to Wilhelm Conrad Röntgen for his discovery of X-rays.\n\nThe Nobel Prize in Phys'}]